<a href="https://colab.research.google.com/github/Interpause/pseudo-text/blob/master/how_to_query_a_database_EX_edition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
%unload_ext google.colab.data_table
%tensorflow_version 2.x
from google.colab import drive #drive.flush_and_unmount()
import os
import numpy as np
import pandas as pd
from collections import OrderedDict
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/CSIT Internship/code')

The google.colab.data_table extension is not loaded.
TensorFlow 2.x selected.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
df = pd.read_pickle('./raw/myNewsFinal.pkl')
#df1.join(df2, how='outer').sort_index(1)

In [0]:
def query(sent,vectorizer,dist,embeds,index,db,top_n=10,reverse=False):
    aquery = vectorizer(sent)
    reply = OrderedDict(sorted(zip(index,dist(aquery,embeds)),key = lambda x: x[1],reverse=reverse))
    for ind,dist in list(reply.items())[:10]:
        print('%s\t%f\t%s'%(str(ind)[:4],dist,db.df.loc[db.df.uuid==ind].title.values[0]))
    print(df.loc[df.uuid==list(reply.keys())[0]].text.values[0])

In [0]:
#@title TS_SS stuff {display-mode: "form"}
#@markdown https://github.com/taki0112/Vector_Similarity (i stole this)
class TS_SS:
    def Cosine(self, vec1: np.ndarray, vec2: np.ndarray):
        return np.dot(vec1, vec2.T)/(np.linalg.norm(vec1) * np.linalg.norm(vec2))

    def VectorSize(self, vec: np.ndarray):
        return np.linalg.norm(vec)

    def Euclidean(self, vec1: np.ndarray, vec2: np.ndarray):
        return np.linalg.norm(vec1-vec2)

    def Theta(self, vec1: np.ndarray, vec2: np.ndarray):
        return np.arccos(self.Cosine(vec1, vec2)) + np.radians(10) #apparently the "best" value to solve overlapping problem (seems arbitrary tho)

    def Triangle(self, vec1: np.ndarray, vec2: np.ndarray):
        theta = np.radians(self.Theta(vec1, vec2))
        return (self.VectorSize(vec1) * self.VectorSize(vec2) * np.sin(theta))/2

    def Magnitude_Difference(self, vec1: np.ndarray, vec2: np.ndarray):
        return abs(self.VectorSize(vec1) - self.VectorSize(vec2))

    def Sector(self, vec1: np.ndarray, vec2: np.ndarray):
        ED = self.Euclidean(vec1, vec2)
        MD = self.Magnitude_Difference(vec1, vec2)
        theta = self.Theta(vec1, vec2)
        return np.pi * (ED + MD)**2 * theta/360

    def __call__(self, vec1: np.ndarray, vec2: np.ndarray):
        return self.Triangle(vec1, vec2) * self.Sector(vec1, vec2)
tsss = TS_SS()

In [0]:
import sklearn.metrics
def cosine(vec1,samples):
    ans = sklearn.metrics.pairwise.cosine_similarity(np.array([vec1]),samples)
    return 1 - ans[0]

In [0]:
import functools
def distwrap(f):
    @functools.wraps(f)
    def dist(query,docs):
        return 1 - f(np.array([query]),docs)[0]
    return dist

@distwrap
def dis(docs1,docs2):
    return sklearn.metrics.pairwise.cosine_similarity(docs1,docs2)
#just a little fun smth. nice to know it actually worked

# Future approaches
- BERTsumEXT sentences before embedding
- Factor in date

##TODO:
- Try hashingVectorizer?
- bertEXT instead of textRank
- gensim sent vec, doc vec instead of sent-transformers
- Try out flair's stacked embeddings
- spacy's NER? flair's NER? well it will be needed for active scrape
- NER based search approach
- Try LDA. the idea of a probability distribution a document should be in the same genre... sounds smart

#TFIDF Base

In [7]:
import string
import spacy
import nltk
from collections import OrderedDict
nltk.download('stopwords')
removables = nltk.corpus.stopwords.words('english')
replacements = OrderedDict({'barack obama':'obama','donald trump':'trump','health care':'healthcare','hillary clinton':'hillary','president obama':'obama','barack':'obama','donald':'trump','gov ':'government ','said':'say'})
class LemmaTokenizer(object):
    def __init__(self):
        self.nlp = spacy.load('en', disable=['parser', 'ner'])
    def __call__(self, articles):
        for rep,val in replacements.items(): articles = articles.replace(rep,val)
        return [str(t.lemma_) for t in self.nlp(articles) if t.text not in removables]
lemma = LemmaTokenizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import OrderedDict
text_vectorizer = TfidfVectorizer(lowercase=True,
                            #tokenizer=lemma, #lemmatizer somehow accents word sense disambiguation problems
                            stop_words='english',
                            ngram_range=(1,2),
                            max_features=25000,
                            norm=None)
title_vectorizer = TfidfVectorizer(lowercase=True,
                            #tokenizer=lemma, #lemmatizer somehow accents word sense disambiguation problems (well it makes sense that if you reduce words to root forms...)
                            stop_words='english',
                            ngram_range=(1,2),
                            max_features=25000,
                            norm=None)
#TODO, increase ngram_range since word2vecs support phrases (underscores i think?)

In [0]:
database = pd.concat({'df':df.copy(),
                 'tfidf_text':pd.DataFrame.sparse.from_spmatrix(text_vectorizer.fit_transform(df.text),index=df.index),
                 'tfidf_title':pd.DataFrame.sparse.from_spmatrix(title_vectorizer.fit_transform(df.title),index=df.index)
                 },
                axis=1)#,copy=False)

In [0]:
test_query = 'vaccines cause autism'
print('Cosine text')
query(test_query,
      lambda s: text_vectorizer.transform([s]).toarray()[0],
      cosine,
      database.tfidf_text.values,
      database.df.uuid,
      database)
print('\nTS-SS text')
query(test_query,
      lambda s: text_vectorizer.transform([s]).toarray()[0],
      tsss,
      database.tfidf_text.values,
      database.df.uuid,
      database)
print('\nCosine title')
query(test_query,
      lambda s: title_vectorizer.transform([s]).toarray()[0],
      cosine,
      database.tfidf_title.values,
      database.df.uuid,
      database)
print('\nTS-SS title')
query(test_query,
      lambda s: title_vectorizer.transform([s]).toarray()[0],
      tsss,
      database.tfidf_title.values,
      database.df.uuid,
      database)

Cosine text
245b	0.607406	i'm thankful for autism, eight reasons this thanksgiving i am giving thanks
ba97	0.652245	autism and adhd share genes
aeb0	0.660046	vaccines fast facts
946e	0.729204	a parent-to-parent campaign to get vaccine rates up
ecfd	0.729218	her son died. and then anti-vaxers attacked her
56cf	0.741302	autism news
7a57	0.774695	autism news
14a6	0.809315	samoa measles deaths rise above 50, mostly small children
6aa5	0.809315	samoa measles deaths rise above 50, mostly small children
312a	0.814439	samoa measles deaths rise above 50, mostly small children
i never really thought i would utter the words "i'm thankful for autism" though i have and while i certainly have moments of self-doubt, feelings of being overwhelmed, wants of wishing life was easier for my daughter, there are so many things that i am thankful for, because of autism. first, autism has allowed me to gain true friends. choosing to be a friend to a person or with a family who lives with a loved one on the sp

## Brief analysis
TS-SS seems better than cosine on average. (probably not confirmation bias). Title seems better than text on average too but perhaps thats me judging a book by the cover.

In [0]:
import pickle
with open('./tfidf_vectorizer.bin','wb') as f:
    pickle.dump({
        'tfidf_text':text_vectorizer,
        'tfidf_title':title_vectorizer
    },f)

# Reload Vectorizers

In [0]:
import pickle
with open('./tfidf_vectorizer.bin','rb') as f:
    temp = pickle.load(f)
    text_vectorizer = temp['tfidf_text']
    title_vectorizer = temp['tfidf_title']
    del temp

In [0]:
database = pd.concat({'df':df.copy(),
                 'tfidf_text':pd.DataFrame.sparse.from_spmatrix(text_vectorizer.transform(df.text),index=df.index),
                 'tfidf_title':pd.DataFrame.sparse.from_spmatrix(title_vectorizer.transform(df.title),index=df.index)
                 },
                axis=1)#,copy=False)

# tfidf + GloVe + tsss

In [0]:
class vectorGetter():
    def __init__(self,word2vec,vocab2ind):
        self.word2vec = word2vec
        self.ind2vocab = {i:w for w,i in vocab2ind.items()}
        self.vec_dim = word2vec.vector_size

    def __call__(self,t):
        w = '_'.join(self.ind2vocab[t].split())
        try:
            emb = self.word2vec[w]
        except KeyError:
            embs = []
            try:
                for sw in w.split('_'): embs.append(self.word2vec[sw])
                emb = np.mean(embs,axis=0)
            except KeyError:
                return np.zeros(self.vec_dim,dtype=np.float32)
        return emb

In [0]:
from sklearn.preprocessing import normalize
class embedtsss(vectorGetter):
    def __init__(self,word2vec,vocab2ind):
        super(embedtsss,self).__init__(word2vec,vocab2ind)
        #self.tsss = TS_SS()
        self.tsss = cosine

    def convert_tfidf(self,w):
        con = {i:{} for i in range(len(w))}
        for y,x in np.column_stack(np.nonzero(w)):
            con[y][x] = w[y][x]
        return con

    def gen_weights(self,docs):
        weights = np.zeros((len(docs),self.vec_dim),dtype=np.float32)
        for i,freq in docs.items():
            for t,f in freq.items():
                weights[i] += f * super().__call__(t)
        return weights
    
    def __call__(self,query,samples):
        dists = []
        doc1 = self.convert_tfidf(normalize((query,),norm='l2'))
        docs = self.convert_tfidf(normalize(samples,norm='l2'))
        w1 = self.gen_weights(doc1)[0]
        w2 = self.gen_weights(docs)
        return self.tsss(w1,w2)

In [0]:
test_query = 'vaccines cause autism'
etsss1 = embedtsss(glove,text_vectorizer.vocabulary_)
etsss2 = embedtsss(glove,title_vectorizer.vocabulary_)
#^recalculated weights werent cached...
print('TS-SS text')
query(test_query,
      lambda s: text_vectorizer.transform([s]).toarray()[0],
      etsss1, #i changed from tsss to cosine in the class declaration temporarily
      database.tfidf_text.values,
      database.df.uuid,
      database)
'''
print('\nTS-SS title')
query(test_query,
      lambda s: title_vectorizer.transform([s]).toarray()[0],
      etsss2,
      database.tfidf_title.values,
      database.df.uuid,
      database)
'''

TS-SS text
ba97	0.225629	autism and adhd share genes
1f58	0.261112	vaccines news
140d	0.268623	bird flu research news
0984	0.273196	bird flu news
2028	0.277309	influenza news
a2a3	0.279930	japan launches human trial of new ebola vaccine
9275	0.283978	malaysias first polio case in 27 years is a child who wasnt vaccinated. here are 3 things to know about the incurable virus
78a1	0.284980	new vaccine will stop the spread of bovine tb
0333	0.285415	building a better flu shot
9380	0.287054	polio scare: malaysia begins vaccination campaign as first case in 27 years reported
researchers from the national psychiatric project ipsych have found that autism and adhd share changes in the same genes. the new knowledge relates directly to the biological causes of the two child psychiatric disorders. in denmark, approximately one per cent of schoolchildren have autism and two to three per cent are diagnosed with adhd. autism and adhd are different developmental disorders, but they can have certain co

"\nprint('\nTS-SS title')\nquery(test_query,\n      lambda s: title_vectorizer.transform([s]).toarray()[0],\n      etsss2,\n      database.tfidf_title.values,\n      database.df.uuid,\n      database)\n"

Too general. L2 norm was very general, L1 norm was slightly better... how about max norm? okay it was much worse. Perhaps it has to do with naively comparing distances between relatively low-dimensional word embeddings (seen from quite low distances when using cosine). Question then would be if so, why would the same approach but using WMD instead be so successful?

Cosine distance is better than tsss in this case. Cosine L2 better than cosine L1 or max? what?

# tfidf + GloVe + WMD

In [0]:
#!pip install wmd
import wmd
#^needed to get libwmdrelax
import libwmdrelax
class wmdistance(vectorGetter):
    def __init__(self,word2vec,vocab2ind):
        super(wmdistance,self).__init__(word2vec,vocab2ind)
    
    def convert_tfidf(self,w):
        return {t:w[t] for t in np.nonzero(w)[0]}

    def get_vocab(self,doc1,doc2):
        return {t:i for i,t in enumerate(sorted(set(doc1).union(doc2)))}

    def gen_weights(self,doc,vocab):
        w = np.zeros(len(vocab),dtype=np.float32)
        if(len(doc) == 0): return w
        for t,f in doc.items():
            w[vocab[t]] = f
        w /= w.sum()
        return w

    def __call__(self,query,samples):
        dists = []
        doc1 = self.convert_tfidf(query)
        for doc in samples:
            doc2 = self.convert_tfidf(doc)
            dists.append(self.calc_dist(doc1,doc2))
        return dists
    
    #In summary, for only the set of words present in the compared docs:
    # weight of words determined by input
    # distance between words calculated from embeddings
    # calculates earth mover's distance using the above
    # as tfidf may have been too loose for keyword extraction
    # Key is to strategically deweigh/remove unimportant words
    # ^find better weighing system?
    def calc_dist(self,doc1,doc2):
        vocab = self.get_vocab(doc1,doc2)
        w1 = self.gen_weights(doc1, vocab)
        w2 = self.gen_weights(doc2, vocab)
        if np.all(w1==0) or np.all(w2==0): return np.inf
        evec = np.zeros((len(vocab),self.vec_dim),dtype=np.float32)
        for t,i in vocab.items():
            evec[i] = super().__call__(t)
        evec_sqr = (evec*evec).sum(axis=1)
        dists = evec_sqr - 2*evec.dot(evec.T)+evec_sqr[:, np.newaxis]
        dists[dists < 0] = 0
        dists = np.sqrt(dists)
        try:
            ans = libwmdrelax.emd(w1,w2,dists)
        except RuntimeError:
            print(dists)
            print(w1)
            print(w2)
            raise RuntimeError
        return ans

In [0]:
wmd1 = wmdistance(glove,text_vectorizer.vocabulary_)
#cannot cache weights, wmd weights dependent on context of both docs. At most find faster version of np.nonzero given that sparse matrix is used
query('vaccines cause autism',
      lambda s: text_vectorizer.transform([s]).toarray()[0],
      wmd1,
      database.tfidf_text.values,
      database.df.uuid,
      database)

ba97	0.991549	autism and adhd share genes
0984	1.050522	bird flu news
1f58	1.055506	vaccines news
0333	1.062309	building a better flu shot
140d	1.062313	bird flu research news
90fc	1.067561	cancer patients, survivors face increased risk of heart disease deaths
524d	1.068516	add and adhd news
b76a	1.071482	building a better flu vaccine -- one you don't have to get every year
1fd4	1.071544	the 2019-2020 flu season is off to an early start
c21e	1.071725	infectious diseases news
researchers from the national psychiatric project ipsych have found that autism and adhd share changes in the same genes. the new knowledge relates directly to the biological causes of the two child psychiatric disorders. in denmark, approximately one per cent of schoolchildren have autism and two to three per cent are diagnosed with adhd. autism and adhd are different developmental disorders, but they can have certain common symptoms. for example, children with autism can demonstrate violent or aggressive behaviou

In [0]:
wmd2 = wmdistance(glove,title_vectorizer.vocabulary_)
query('vaccines cause autism',
      lambda s: title_vectorizer.transform([s]).toarray()[0],
      wmd2,
      database.tfidf_title.values,
      database.df.uuid,
      database)

7a57	0.681598	autism news
56cf	0.681598	autism news
1f58	0.690510	vaccines news
aeb0	0.765396	vaccines fast facts
ba97	0.806746	autism and adhd share genes
a2dd	0.876639	nurse takes in man with autism so he can get a new heart
78a1	0.912511	new vaccine will stop the spread of bovine tb
b76a	0.951569	building a better flu vaccine -- one you don't have to get every year
245b	0.952104	i'm thankful for autism, eight reasons this thanksgiving i am giving thanks
ccbd	0.961914	side effects of pediatric medications for anxiety, ocd
read more down syndrome, due to an extra chromosome 21, occurs in 250,000 children and adults in the united states, making it the country's most common chromosomal disorder. inherited heart defects, ... read more a research team has found exposure to sound -- not sound reduction -- during early development of mice engineered to have fragile x syndrome, or fxs, restores molecular, cellular, and functional ... read more the annual transition to and from daylight savin

## Brief Analysis
Even only using global vectors (static), seems to be extremely on point with queries... except it took forever to run (for text at least). Try compiling it? Well, seems definitely much more on point then the other base tfidf text. Should be better on tfidf title since title is better topic indicator (see previous analysis). however, it did miss the really good article tfidf ts-ss text recommended as first.

Title processed faster but seems to only be on par. So I guess tfidf title first, and then wmd text (though it takes forever). I have yet to try NER.

On better understanding of how WMD works, as tfidf may have been too loose for keyword extraction, strategically deweighing/removing unimportant words using other techniques (NER?) might help calculate better distances.

# Sent2Vec Base

In [0]:
#!pip install -U sentence-transformers
promising = (
#'bert-base-wikipedia-sections-mean-tokens', #crap
'distilbert-base-nli-stsb-mean-tokens',
'roberta-base-nli-stsb-mean-tokens',
#'roberta-base-nli-mean-tokens', #crap
#'distilbert-base-nli-mean-tokens'
)
from sentence_transformers import SentenceTransformer

In [0]:
from gensim.summarization import summarize, keywords
def encode_document(sent,model):
    return model.encode([sent])[0]
    #try:
        #ext = summarize(sent,split=True)
        #if len(ext) <= 1:
        #    return model.encode([sent])[0]
        #else:
        #    return np.mean(model.encode(ext),axis=0)
    #except ValueError:
    #    return model.encode([sent])[0]
def encode_documents(sents,model):
    return np.stack(pd.Series(sents).apply(encode_document,args=(model,)))
#np.mean(model.encode(summarize(df.loc[31].text,split=True)),axis=0)

In [0]:
all_enc = {}
for model_name in promising:
    model = SentenceTransformer(model_name)
    all_enc[model_name+'_title'] = pd.DataFrame(encode_documents(df.title,model),index=df.index)
    all_enc[model_name+'_text'] = pd.DataFrame(encode_documents(df.text,model),index=df.index)

In [0]:
import pickle
with open('./raw/sentTransData.bin','wb') as f:
    pickle.dump(all_enc,f)

In [0]:
import pickle
with open('./raw/sentTransData.bin','rb') as f:
    all_enc = pickle.load(f)

In [0]:
all_enc['df'] = df.copy()
db2 = pd.concat(all_enc,axis=1)#,copy=False)

In [0]:
test_query = "vaccines cause autism"
for model_name in promising:
    model = SentenceTransformer(model_name)
    print(model_name)
    print('Title')
    query(test_query,
        lambda s: encode_document(s,model),
        tsss,
        db2[model_name+'_title'].values,
        db2.df.uuid,
        db2)
    print('\nText')
    query(test_query,
        lambda s: encode_document(s,model),
        tsss,
        db2[model_name+'_text'].values,
        db2.df.uuid,
        db2)
    print('\n')

distilbert-base-nli-stsb-mean-tokens
Title
1f58	98861336.000000	vaccines news
7a57	98914192.000000	autism news
56cf	98914192.000000	autism news
aeb0	98951768.000000	vaccines fast facts
2981	99053704.000000	there's a promising new vaccine for one of the world's top health threats
5634	99097680.000000	an experimental genetic test gives early warning for kids at risk of type 1 diabetes
ba97	99102272.000000	autism and adhd share genes
ccbd	99142496.000000	side effects of pediatric medications for anxiety, ocd
2028	99154816.000000	influenza news
5f06	99155096.000000	specialized immune cells could help repair inflammatory bowel disease damage in children
read more a new study has found severe pneumonia decreases by 35 per cent in children who receive a vaccine against a pneumonia-causing ... read more a newly developed experimental vaccine was more than eighty percent effective in protecting mice from succumbing to staphylococcus aureus infection. s. aureus causes more than 30,000 deaths fro

## Brief Analysis
Actually loses out to tfidf based on this search query. Seems more capable than tfidf at gathered articles that share similar contexts - except they are too broad. For example, using the PIE expressway query, it doesn't pull up results related to pie crusts. Corroborates that a tfidf weighted approach might be best.

# GENSIM
used for word embeddings (tfidf as weights approach), provides distance formulae like cosine and WMD

In [0]:
import gensim
from gensim.models import KeyedVectors
#word_vectors.distance("media", "media") #cosine

## Conversion code for saving ignore if already run

In [0]:
#model.syn0norm = model.syn0
#newsve.init_sims(replace=True)
#glove.init_sims(replace=True)

In [0]:
#newsve.save('GoogleNews-vectors-gensim-normed.bin')
#glove.save('gensim_glove_vectors_normed.bin')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


## Load vectors

In [0]:
glove_loc = '/content/drive/My Drive/CSIT Internship/code/gensim_glove_vectors_normed.bin' #Common crawl uncased 42 billion tokens 300 dimensions
word2vec_loc = '/content/drive/My Drive/CSIT Internship/code/GoogleNews-vectors-gensim-normed.bin' #google news 3 million tokens 300 dimensions

In [0]:
#newsve = KeyedVectors.load(word2vec_loc, mmap='r')
#newsve.syn0norm = newsve.syn0
glove = KeyedVectors.load(glove_loc, mmap='r')
glove.syn0norm = glove.syn0


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
  


# Flair Embedddings


In [13]:
#!pip install https://github.com/zalandoresearch/flair/archive/GH-1221-fix-tokenization-problems.zip
from flair.embeddings import DocumentPoolEmbeddings,WordEmbeddings, Sentence, XLNetEmbeddings
import torch
torch.cuda.empty_cache()
# init embedding
xlnet_embedding = XLNetEmbeddings("xlnet-base-cased")#, layers="0,1,2,3,4,5,6,7,8,9,10,11,12",pooling_operation="first", use_scalar_mix=True)
#flair_forward_embedding = FlairEmbeddings('mix-forward')
#flair_backward_embedding = FlairEmbeddings('mix-backward')
base_embedding = WordEmbeddings('en') #fasttext without subwords
#bpe_embedding = BytePairEmbeddings('en')
#roberta_embedding = RoBERTaEmbeddings('roberta-base')

#and henceforth, Im doomed to mix and match for all of friday.
#https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/discussion/90426

stacked_embedder = DocumentPoolEmbeddings(embeddings=[xlnet_embedding, base_embedding])

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
def temp1(s):
    temp = Sentence(s)
    #print(temp)
    try:
        stacked_embedder.embed(temp)
        ans = temp.embedding.cpu().detach().numpy()
    except RuntimeError:
        torch.cuda.empty_cache()
        try:
            stacked_embedder.embed(temp)
            ans = temp.embedding.cpu().detach().numpy()
        except RuntimeError:
            ans = np.zeros(stacked_embedder.embedding_length,dtype=np.float32)
            torch.cuda.empty_cache()
            print(s)
    del temp
    return ans

In [0]:
db3 = pd.concat({'df':df.copy(),
                 'flair_text':pd.DataFrame(np.stack(df.text.apply(temp1).values),index=df.index),
                 'flair_title':pd.DataFrame(np.stack(df.title.apply(temp1).values),index=df.index)
                 },
                axis=1)#,copy=False)
#^ for the record this took approximately more than 3 hours [flair forward, backward, XLNET, base]
#^ 11 mins for [XLnNet, Base]
db3.to_pickle('./raw/db3.pkl')
#tends to drop sentences. use a empty_cache, retry, on fail give up style

In [0]:
db3 = pd.read_pickle('./raw/db3.pkl')

In [16]:
test_query = Sentence('vaccines cause autism')
stacked_embedder.embed(test_query)
print('Cosine text')
query(test_query,
      lambda s: test_query.embedding.cpu().detach().numpy(),
      cosine,
      db3.flair_text.values,
      db3.df.uuid,
      db3)
print('\nTS-SS text')
query(test_query,
      lambda s: test_query.embedding.cpu().detach().numpy(),
      tsss,
      db3.flair_text.values,
      db3.df.uuid,
      db3)
print('\nCosine title')
query(test_query,
      lambda s: test_query.embedding.cpu().detach().numpy(),
      cosine,
      db3.flair_title.values,
      db3.df.uuid,
      db3)
print('\nTS-SS title')
query(test_query,
      lambda s: test_query.embedding.cpu().detach().numpy(),
      tsss,
      db3.flair_title.values,
      db3.df.uuid,
      db3)

Cosine text
64db	0.614733	programmes in place at public hospitals to counter antibiotic resistance
9fae	0.620411	concussion recovery not clear cut for children
aeb0	0.629850	vaccines fast facts
c968	0.642287	dementia study reveals how proteins interact to stop brain signals
0333	0.644414	building a better flu shot
140d	0.648763	bird flu research news
1f58	0.649355	vaccines news
4ffa	0.649600	type 2 diabetes: drinking this type of tea could lower blood sugar
fa8d	0.650068	genetic brain disorder fixed in mice using precision epigenome editing
bcf9	0.651480	california considers calling thc in pot a risk to moms-to-be
we are pleased to see the increasing awareness of antibiotic resistance and applaud the study by the singapore general hospital on not using antibiotics to treat non-bacterial infection (stopping antibiotics early may help patients: sgh, nov 27). antibiotics work on bacteria by stopping their growth or killing them. they do not work on infections caused by viruses. the basic 

Embedding TS-SS title seems conciously aware that it is looking for fact pages without being indicated to do so somehow. Other than, accuracy is pretty low, a lot of other health articles being mixed in. Perhaps the usage of flair embedders, which encode structure of sentences, without one more layer to interpret that properly into the vector, distorts the results because now sentence with similar structure, though different meanings, are evaluated higher.

After testing without Flair embeddings, it does not seem to have gotten better. Perhaps finetuning & investigating suitable output layer is necessary. Alternatively, what is word mover's distance was added to this?